In [1]:
import os
import json
import hashlib
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

MERGED_CSV = Path("./data/b365_merged.csv")
RAW_DIR = Path("./data")

In [2]:
merged = pd.read_csv(MERGED_CSV, parse_dates=["Date"], dayfirst=True)
merged = merged.sort_values(["Date","Div","HomeTeam","AwayTeam"]).drop_duplicates(subset=["match_id"], keep="first").reset_index(drop=True)

/var/folders/xy/fs1cvf415szgw9x79lyywp1h0000gn/T/ipykernel_8112/546588977.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv(MERGED_CSV, parse_dates=["Date"], dayfirst=True)


In [3]:
merged

,match_id,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HC,...,HTHG,HTAG,HTR,Attendance,Referee,__source_file,jn_date,jn_home,jn_away,jn_key
0,3ee520466726d91f,F1,NaN,2015-08-07,Lille,Paris SG,0.0,1.0,A,3.0,...,0.0,0.0,D,NaN,NaN,F1 (10).csv,2015-08-07,lille,paris sg,2015-08-07|lille|paris sg
1,92ff78b3164950d0,E0,NaN,2015-08-08,Bournemouth,Aston Villa,0.0,1.0,A,6.0,...,0.0,0.0,D,NaN,M Clattenburg,E0 (10).csv,2015-08-08,bournemouth,aston villa,2015-08-08|bournemouth|aston villa
2,481f9a32684b2ed6,E0,NaN,2015-08-08,Chelsea,Swansea,2.0,2.0,D,4.0,...,2.0,1.0,H,NaN,M Oliver,E0 (10).csv,2015-08-08,chelsea,swansea,2015-08-08|chelsea|swansea
3,5cd2e61c3f58beae,E0,NaN,2015-08-08,Everton,Watford,2.0,2.0,D,8.0,...,0.0,1.0,A,NaN,M Jones,E0 (10).csv,2015-08-08,everton,watford,2015-08-08|everton|watford
4,c0470cbc33f6e441,E0,NaN,2015-08-08,Leicester,Sunderland,4.0,2.0,H,6.0,...,3.0,0.0,H,NaN,L Mason,E0 (10).csv,2015-08-08,leicester,sunderland,2015-08-08|leicester|sunderland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18236,e6e2c60f41c2acd4,NaN,NaN,2025-09-23,Sevilla,Villarreal,1.0,2.0,A,3.0,...,0.0,1.0,A,NaN,NaN,SP1.csv,2025-09-23,sevilla,villarreal,2025-09-23|sevilla|villarreal
18237,ebe524fb650ef71e,NaN,NaN,2025-09-24,Ath Madrid,Vallecano,3.0,2.0,H,8.0,...,1.0,1.0,D,NaN,NaN,SP1.csv,2025-09-24,ath madrid,vallecano,2025-09-24|ath madrid|vallecano
18238,132df5db35d608fd,NaN,NaN,2025-09-24,Getafe,Alaves,1.0,1.0,D,2.0,...,0.0,0.0,D,NaN,NaN,SP1.csv,2025-09-24,getafe,alaves,2025-09-24|getafe|alaves
18239,bbcb70a8b61fc611,NaN,NaN,2025-09-24,Sociedad,Mallorca,1.0,0.0,H,11.0,...,0.0,0.0,D,NaN,NaN,SP1.csv,2025-09-24,sociedad,mallorca,2025-09-24|sociedad|mallorca


In [4]:

data = merged.copy()

data["total_corners"] = data["HC"] + data["AC"]

def implied_probs(prices):
    # convert to implied probabilities
    inv = 1.0 / prices
    s = inv.sum(axis=1)
    return inv.div(s, axis=0)

if {"close_home","close_draw","close_away"}.issubset(data.columns):
    probs = implied_probs(data[["close_home","close_draw","close_away"]])
else:
    probs = implied_probs(data[["pre_home","pre_draw","pre_away"]])

data["p_home"] = probs.iloc[:,0]
data["p_draw"] = probs.iloc[:,1]
data["p_away"] = probs.iloc[:,2]

if {"close_over2_5","close_under2_5"}.issubset(data.columns):
    inv_over = 1.0 / data["close_over2_5"]
    inv_under = 1.0 / data["close_under2_5"]
else:
    inv_over = 1.0 / data["pre_over2_5"]
    inv_under = 1.0 / data["pre_under2_5"]
data["p_over25"] = inv_over / (inv_over + inv_under)


model_df = data.dropna(subset=["total_corners","HomeTeam","AwayTeam"])
model_df = model_df[model_df["total_corners"] >= 0]

print("Modeling rows:", model_df.shape[0])
model_df[["Date","Div","HomeTeam","AwayTeam","total_corners","p_home","p_over25"]].head(5)


Modeling rows: 18238


,Date,Div,HomeTeam,AwayTeam,total_corners,p_home,p_over25
0,2015-08-07,F1,Lille,Paris SG,5.0,NaN,NaN
1,2015-08-08,E0,Bournemouth,Aston Villa,9.0,NaN,NaN
2,2015-08-08,E0,Chelsea,Swansea,12.0,NaN,NaN
3,2015-08-08,E0,Everton,Watford,10.0,NaN,NaN
4,2015-08-08,E0,Leicester,Sunderland,9.0,NaN,NaN


In [5]:

df_feat = model_df.copy()
df_feat = df_feat.sort_values("Date").reset_index(drop=True)


# pace ELO rolling window params
win = 20
minp = 5


home_part = df_feat[["Date","HomeTeam","total_corners"]].rename(columns={"HomeTeam":"Team"})
away_part = df_feat[["Date","AwayTeam","total_corners"]].rename(columns={"AwayTeam":"Team"})
long_team = pd.concat([home_part, away_part], ignore_index=True)

long_team = long_team.sort_values(["Team","Date"])
long_team["team_pace"] = (
    long_team
    .groupby("Team")["total_corners"]
    .apply(lambda s: s.shift(1).rolling(win, min_periods=minp).mean())
    .reset_index(level=0, drop=True)
)

home_pace = (long_team.rename(columns={"Team":"HomeTeam"})
                        .set_index(["HomeTeam","Date"])["team_pace"])
away_pace = (long_team.rename(columns={"Team":"AwayTeam"})
                        .set_index(["AwayTeam","Date"])["team_pace"])

df_feat = df_feat.set_index(["HomeTeam","Date"])
df_feat["pace_elo_home"] = home_pace
df_feat = df_feat.reset_index()

df_feat = df_feat.set_index(["AwayTeam","Date"])
df_feat["pace_elo_away"] = away_pace
df_feat = df_feat.reset_index()

df_feat = df_feat.dropna(subset=["pace_elo_home", "pace_elo_away"]).reset_index(drop=True)

# If early-season rows lack history, we can backfill with per-team mean, then global mean
team_means = (
    pd.concat([
        long_team.groupby("Team")["team_pace"].mean().rename("m1"),        # <-- removed min_count
        long_team.groupby("Team")["total_corners"].mean().rename("m2")
    ], axis=1)
    .assign(pace_fallback=lambda d: d["m1"].fillna(d["m2"]))
    ["pace_fallback"]
)

def fill_team_pace(row, col_name, team_col):
    val = row[col_name]
    if pd.notna(val): 
        return val
    tm = row[team_col]
    if tm in team_means:
        return team_means.loc[tm]
    return long_team["total_corners"].mean()

df_feat["pace_elo_home"] = df_feat.apply(lambda r: fill_team_pace(r, "pace_elo_home", "HomeTeam"), axis=1)
df_feat["pace_elo_away"] = df_feat.apply(lambda r: fill_team_pace(r, "pace_elo_away", "AwayTeam"), axis=1)


def elo_z(x):
    x = np.asarray(x, dtype=float)
    return (x - np.nanmean(x)) / (np.nanstd(x) + 1e-8)

df_feat["z_pace_diff"] = elo_z(df_feat["pace_elo_home"] - df_feat["pace_elo_away"])
df_feat["z_pace_sum"]  = elo_z(df_feat["pace_elo_home"] + df_feat["pace_elo_away"])

home_cats = pd.Categorical(df_feat["HomeTeam"])
away_cats = pd.Categorical(df_feat["AwayTeam"], categories=home_cats.categories)
home_idx = home_cats.codes
away_idx = away_cats.codes
n_teams  = len(home_cats.categories)
div_cats = pd.Categorical(df_feat["Div"])
div_idx = div_cats.codes
n_divs  = len(div_cats.categories)

y = df_feat["total_corners"].astype(int).values

print("n_obs:", len(y), "n_teams:", n_teams, "n_divs:", n_divs)

n_obs: 17672 n_teams: 160 n_divs: 5


In [ ]:
with pm.Model() as nb_corners:
    sigma_team = pm.HalfNormal("sigma_team", 1.0)
    sigma_div  = pm.HalfNormal("sigma_div", 1.0)

    intercept = pm.Normal("intercept", 0.0, 1.5)
    home_adv  = pm.Normal("home_adv", 0.0, 0.5)

    team_home_eff = pm.Normal("team_home_eff", 0.0, sigma_team, shape=n_teams)
    team_away_eff = pm.Normal("team_away_eff", 0.0, sigma_team, shape=n_teams)

    div_eff = pm.Normal("div_eff", 0.0, sigma_div, shape=n_divs)

    beta_pace_diff = pm.Normal("beta_pace_diff", 0.0, 0.5)
    beta_pace_sum  = pm.Normal("beta_pace_sum", 0.0, 0.5)

    # linear model from features
    eta = (
        intercept
        + home_adv
        + team_home_eff[home_idx]
        + team_away_eff[away_idx]
        + div_eff[div_idx]
        + beta_pace_diff * df_feat["z_pace_diff"].values
        + beta_pace_sum  * df_feat["z_pace_sum"].values
    )

    mu = pm.math.exp(eta)
    rho = pm.Normal("rho", mu=3.5, sigma=1.5)
    alpha = pm.Deterministic("alpha", pm.math.exp(rho))

    y_obs = pm.NegativeBinomial("y_obs", mu=mu, alpha=alpha, observed=y)

    idata = pm.sample(4000, tune=4000, target_accept=0.95, chains=4, cores=4, random_seed=42)
    idata = pm.sample_posterior_predictive(idata, extend_inferencedata=True)

az.summary(idata, var_names=[
    "intercept","home_adv","sigma_team","sigma_div",
    "beta_pace_diff","beta_pace_sum","alpha"
])

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_team, sigma_div, intercept, home_adv, team_home_eff, team_away_eff, div_eff, beta_pace_diff, beta_pace_sum, rho]


/opt/anaconda3/envs/bayes-corners/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [ ]:

y_pp_da = idata.posterior_predictive["y_obs"]
obs_dim = [d for d in y_pp_da.dims if d not in ("chain", "draw")][0]

y_pp = (
    y_pp_da
    .stack(sample=("chain", "draw"))
    .transpose("sample", obs_dim)
    .values
)

y_pred_mean = y_pp.mean(axis=0)
y_pred_var  = y_pp.var(axis=0, ddof=1)

print(f"OVERALL — mean(obs)={y.mean():.3f}  mean(pred.mean)={y_pred_mean.mean():.3f}")
print(f"OVERALL — var(obs)={y.var(ddof=1):.3f}  mean(pred.var)={y_pred_var.mean():.3f}")

# quick overall PPC plot (unchanged)
az.plot_ppc(idata, group="posterior")
plt.show()

# 2) Helper: predictive interval coverage
def coverage(y_pp_sub: np.ndarray, y_true_sub: np.ndarray, p: float) -> float:
    lo = np.quantile(y_pp_sub, (1-p)/2, axis=0)
    hi = np.quantile(y_pp_sub, 1-(1-p)/2, axis=0)
    return float(np.mean((y_true_sub >= lo) & (y_true_sub <= hi)))

# 3) Per-division summaries + plots
if "Div" not in df_feat.columns:
    df_feat = df_feat.copy()
    df_feat["Div"] = "ALL"

divisions = pd.Categorical(df_feat["Div"])
div_names = list(divisions.categories)

rows = []

# Set this True to save PNGs instead of (or in addition to) showing
SAVE_FIGS = False
FIG_DPI = 110

for div in div_names:
    idx = np.flatnonzero(divisions == div)
    if len(idx) == 0:
        continue

    y_d = y[idx]
    ypm_d = y_pred_mean[idx]
    ypv_d = y_pred_var[idx]
    ypp_d = y_pp[:, idx]   # (samples, n_div_obs)

    # Summaries
    obs_mean = y_d.mean()
    pred_mean = ypm_d.mean()
    obs_var = y_d.var(ddof=1)
    pred_var = ypv_d.mean()
    rmse_mean = float(np.sqrt(np.mean((y_d - ypm_d)**2)))

    cov50 = coverage(ypp_d, y_d, 0.50)
    cov80 = coverage(ypp_d, y_d, 0.80)
    cov90 = coverage(ypp_d, y_d, 0.90)
    cov95 = coverage(ypp_d, y_d, 0.95)

    rows.append({
        "Division": div,
        "n": len(idx),
        "obs_mean": obs_mean, "pred_mean": pred_mean,
        "obs_var": obs_var,   "pred_var": pred_var,
        "RMSE_mean": rmse_mean,
        "cov50": cov50, "cov80": cov80, "cov90": cov90, "cov95": cov95
    })

    # --- Plots for this division ---
    # A) Observed vs Predicted mean (per match)
    plt.figure(figsize=(5.2, 4.2))
    plt.scatter(y_d, ypm_d, alpha=0.35, s=14)
    lo = min(y_d.min(), ypm_d.min())
    hi = max(y_d.max(), ypm_d.max())
    plt.plot([lo, hi], [lo, hi], lw=1)
    plt.xlabel("Observed total corners")
    plt.ylabel("Predicted mean total corners")
    plt.title(f"{div} — Obs vs Pred mean (n={len(idx)})\n"
              f"mean: {obs_mean:.2f}/{pred_mean:.2f} | var: {obs_var:.2f}/{pred_var:.2f}")
    plt.tight_layout()
    if SAVE_FIGS:
        plt.savefig(f"ppc_mean_scatter_{div}.png", dpi=FIG_DPI)
    plt.show()

    # B) Residuals histogram
    residuals = y_d - ypm_d
    plt.figure(figsize=(5.2, 4.0))
    plt.hist(residuals, bins=20, alpha=0.8)
    plt.axvline(0, color="k", lw=1)
    plt.xlabel("Residual = Observed − Predicted mean")
    plt.ylabel("Count")
    plt.title(f"{div} — Residuals")
    plt.tight_layout()
    if SAVE_FIGS:
        plt.savefig(f"ppc_residuals_{div}.png", dpi=FIG_DPI)
    plt.show()

# 4) Display division summary table (sorted by sample size)
div_summary = pd.DataFrame(rows).sort_values("n", ascending=False)
display(div_summary)



In [ ]:
# === Observed vs Model: mean, variance, coverage, and grouped diagnostics ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) Pull observed and posterior predictive draws
# idata: PyMC v4 InferenceData with posterior_predictive["y_obs"] (chain, draw, obs)
y_true = np.asarray(df_feat["total_corners"].astype(int).values)
y_pp_da = idata.posterior_predictive["y_obs"]

# Convert to numpy with shape (samples, obs)
y_pp = (y_pp_da
        .stack(sample=("chain","draw"))
        .transpose("sample","y_obs_dim_0")
        .values)

# Pointwise predictive mean & variance
mu_hat = y_pp.mean(axis=0)           # (obs,)
var_hat = y_pp.var(axis=0, ddof=1)   # (obs,)

# 2) Overall diagnostics
obs_mean = y_true.mean()
pred_mean = mu_hat.mean()
obs_var  = y_true.var(ddof=1)
pred_var = var_hat.mean()

rmse_mean = float(np.sqrt(np.mean((y_true - mu_hat)**2)))
overdisp_ratio = float(np.var(y_true - mu_hat, ddof=1) / (pred_var + 1e-8))  # 1.0 is ideal-ish

print(f"Overall mean — observed: {obs_mean:.3f} | model: {pred_mean:.3f}")
print(f"Overall var  — observed: {obs_var:.3f}  | model: {pred_var:.3f}")
print(f"RMSE of mean predictions: {rmse_mean:.3f}")
print(f"Residual overdispersion ratio (resid var / avg pred var): {overdisp_ratio:.3f}")

# 3) Coverage checks: does predictive interval contain observed y?
def coverage(p):
    lo = np.quantile(y_pp, (1-p)/2, axis=0)
    hi = np.quantile(y_pp, 1-(1-p)/2, axis=0)
    return float(np.mean((y_true >= lo) & (y_true <= hi)))

for p in (0.50, 0.80, 0.90, 0.95):
    print(f"{int(p*100)}% predictive interval coverage: {coverage(p):.3f}")

# 4) Binned calibration by predicted mean (are we well-calibrated across difficulty?)
bins = pd.qcut(mu_hat, 10, duplicates="drop")
calib = (pd.DataFrame({
            "bin": bins,
            "y_true": y_true,
            "mu_hat": mu_hat,
            "var_hat": var_hat
        })
        .groupby("bin")
        .agg(obs_mean=("y_true","mean"),
             pred_mean=("mu_hat","mean"),
             obs_var=("y_true","var"),
             pred_var=("var_hat","mean"),
             n=("y_true","size"))
        .reset_index())

print("\nCalibration by predicted-mean decile:")
display(calib)

# 5) Grouped diagnostics by league and by home team (customize as needed)
def group_diag(df, group_col):
    g = (pd.DataFrame({
            group_col: df[group_col].values,
            "y_true": y_true,
            "mu_hat": mu_hat,
            "var_hat": var_hat
        })
        .groupby(group_col)
        .agg(obs_mean=("y_true","mean"),
             pred_mean=("mu_hat","mean"),
             obs_var=("y_true","var"),
             pred_var=("var_hat","mean"),
             n=("y_true","size"))
        .sort_values("n", ascending=False))
    return g

if "Div" in df_feat.columns:
    print("\nBy league/division:")
    display(group_diag(df_feat, "Div").head(15))

print("\nBy home team:")
display(group_diag(df_feat, "HomeTeam").head(20))

# 6) Quick visuals (optional)
# Mean calibration plot
plt.scatter(calib["obs_mean"], calib["pred_mean"])
plt.plot([calib["obs_mean"].min(), calib["obs_mean"].max()],
         [calib["obs_mean"].min(), calib["obs_mean"].max()], lw=1)
plt.xlabel("Observed mean (bin)"); plt.ylabel("Predicted mean (bin)")
plt.title("Calibration: mean (by predicted-mean decile)")
plt.show()

# Variance calibration plot
plt.scatter(calib["obs_var"], calib["pred_var"])
plt.plot([calib["obs_var"].min(), calib["obs_var"].max()],
         [calib["obs_var"].min(), calib["obs_var"].max()], lw=1)
plt.xlabel("Observed variance (bin)"); plt.ylabel("Predicted variance (bin)")
plt.title("Calibration: variance (by predicted-mean decile)")
plt.show()


In [ ]:
# output model artifacts


train_feat = df_feat.copy()

ARTIFACT_PREFIX = "/corners_model"  # <- CHANGE THIS
Path(os.path.dirname(ARTIFACT_PREFIX)).mkdir(parents=True, exist_ok=True)

def make_team_universe(df):
    ht = df["HomeTeam"].astype(str).unique()
    at = df["AwayTeam"].astype(str).unique()
    return sorted(set(ht) | set(at))

def make_div_universe(df):
    if "Div" in df.columns:
        return sorted([d for d in df["Div"].dropna().astype(str).unique()])
    return []

team_universe = make_team_universe(train_feat)
div_universe  = make_div_universe(train_feat)

def safe_stats(series, default_mean=0.5, default_std=0.1):
    if series is None:
        return float(default_mean), float(default_std)
    x = pd.to_numeric(series, errors="coerce")
    m = float(np.nanmean(x)) if np.isfinite(np.nanmean(x)) else default_mean
    s = float(np.nanstd(x))  if np.isfinite(np.nanstd(x))  else default_std
    s = max(s, 1e-6)
    return m, s

if "p_home" in train_feat.columns:
    mean_p_home, std_p_home = safe_stats(train_feat["p_home"])
else:
    mean_p_home, std_p_home = 0.5, 0.1
    print("NOTE: 'p_home' not found; using defaults mean=0.5, std=0.1")

if "p_over25" in train_feat.columns:
    mean_p_over25, std_p_over25 = safe_stats(train_feat["p_over25"])
else:
    mean_p_over25, std_p_over25 = 0.5, 0.1
    print("NOTE: 'p_over25' not found; using defaults mean=0.5, std=0.1")

zstats = {
    "mean_p_home":   float(mean_p_home),
    "std_p_home":    float(std_p_home),
    "mean_p_over25": float(mean_p_over25),
    "std_p_over25":  float(std_p_over25),
}

post = idata.posterior

def flat(name):
    if name not in post:
        return None
    arr = np.asarray(post[name])  # dims: (chain, draw, ...)
    return arr.values.reshape(-1, *arr.shape[2:]) if hasattr(arr, "values") else arr.reshape(-1, *arr.shape[2:])

draws = {}
required_keys = ["intercept", "home_adv", "team_home_eff", "team_away_eff", "alpha"]
optional_keys = ["div_eff", "beta_homeprob", "beta_over25"]

for k in required_keys + optional_keys:
    arr = flat(k)
    if arr is not None:
        # squeeze 0-d where appropriate
        draws[k] = np.squeeze(arr)
    elif k in required_keys:
        raise KeyError(f"Missing required posterior variable '{k}' in idata_tr.posterior")


np.savez_compressed(ARTIFACT_PREFIX + "_draws.npz", **draws)

with open(ARTIFACT_PREFIX + "_teams.json", "w", encoding="utf-8") as f:
    json.dump(list(team_universe), f, ensure_ascii=False)

with open(ARTIFACT_PREFIX + "_divs.json", "w", encoding="utf-8") as f:
    json.dump(list(div_universe), f, ensure_ascii=False)

with open(ARTIFACT_PREFIX + "_zstats.json", "w", encoding="utf-8") as f:
    json.dump(zstats, f)

print("Saved:")
print(" -", ARTIFACT_PREFIX + "_draws.npz")
print(" -", ARTIFACT_PREFIX + "_teams.json")
print(" -", ARTIFACT_PREFIX + "_divs.json")
print(" -", ARTIFACT_PREFIX + "_zstats.json")

